#Tülomsaş Train Traction Motor

##Ratings

- **Power :** 142-152 kW
- **Voltage :** 800-1100 V
- **Speed :** 374- 3120 rpm
- **Mass:** 1560 kg
- **Motor type:** Three phase IM

[Here](http://www.tulomsas.com.tr/main.php?kid=198&rid=101) you can find the related info about Tülomsaş's motor. 

![motor_picture](http://www.tulomsas.com.tr/upload/resim/101.jpg)

####Breakdown torque is:

\begin{equation}
  T_{bk} = \frac{3p_{1}}{2L_{SC}}(\frac{V_{ph}}{\omega_{1}})^2
   ;L_{SC} = L_{ls}+L_{lr}
\end{equation}

where :
- $p_{1}$ is number of pole-pair,i.e $2p_{1}$ equals to total number of poles 
- $V_{ph}$ is phase voltage
- $L_{SC}$ is total leakage inductance
- $L_{ls}$ and $L_{lr}$ are the stator and rotor leakage inductances,respectively


Since the phase voltages and synchronous speeds are variable, constant V/f technique can be used to keep the max. torque constant during various speeds. Thus, in this design parameters are calculated according to one voltage and frequency which are in between the limit values.

- ##Stator Core

In [29]:
from math import sqrt,sin,cos,pi,tan
Pn=150e3 #Power
pf= 0.85 #Power-factor cos(phi) from Fig. 16.1
eff = 0.93 #targeted efficiency
n_s=1250 #synchronous speed
Vln=950 #Line-neutral voltage for delta connection
Vph = Vln #phase voltage
m=3 #number of phase
freq=50 #supply frequency
poles=(120*freq)/n_s #number of poles
pp=poles/2 #number of pole pairs
ke= 0.98-0.005*pp #from eq. 15.1
print poles
print ke

4
0.97


**Number of poles** is calculated as $4$

In [30]:
Sgap=ke*Pn/(eff*pf) #Apparent airgap power 
print Sgap
C_0=235e3 # Esson's constant (J/m^3) from fig. 14.14
lmbd=1.4 # Stack aspect ratio from table 15.1
Dis=((2*poles*pp*Sgap)/(pi*lmbd*freq*C_0))**(1/3.0) #stator bore diameter
print Dis

184060.721063
0.384817761458


**Apparent airgap power** is calculated as $184.06 kVA$ 

Therefore **stator bore diameter** is obtained as $0.384  m$

**Slot pitch** is calculated as $0.025 m$

In [31]:
tau=pi*Dis/(2*pp) #pole pitch
L=lmbd*tau #stack lenght
q=4 #slots per pole per phase value(larger q gives better performance-smaller space field harmonics and losses)
tau_s=tau/(m*q) #slot pitch 
print tau
print L
print tau_s

0.302235163092
0.423129228329
0.025186263591


In [32]:
import math
K_D= 0.62 #selected from table 15.2 for 4 poles
Dout=Dis/K_D #external stator diameter
print Dout
g=(0.1+0.012*math.pow(Pn,1.0/3.0))*1e-3 #air-gap 
print g

0.620673808804
0.00073759514151


**Outer diameter** is calculated as $0.62 m$

**air-gap** value for pole number $\geq 2$ is calculated by using eq.15.5


- ##Stator Winding

In [33]:
Ns=2*pp*q*m #Number of stotor slots
print Ns


48


Two-layer winding with chorded coils, y/$\tau$=7/9 is chosen as 7/9=0.77 is close to 0.8, which would reduce the first (5$^{th}$ order) stator mmf space harmonic.

In [34]:
alpha_e=(2*pi*pp)/Ns
print alpha_e

0.261799387799


Electrical angle between emfs in neighboring slots is shown above cell by *alpha_e* 

it equals to:

##$\alpha_{ec} = \frac{2*\pi*p1}{N_{s}}= \frac{2\pi2}{48} = \frac{\pi}{12}$


In [35]:
K_q1=sin(q*alpha_e/2.0)/(q*sin(alpha_e/2.0)) #distribution factor or zone factor from eq. 15.8
print K_q1
K_y1=sin(pi/2.0*7/9) #pitch factor or chording factor from eq. 15.9
print K_y1
K_w1=K_q1*K_y1 # stator winding factor
print K_w1

0.957662196943
0.939692620786
0.899908099673


$B_{g}=0.72 \ sample \ air \ gap \ flux \ density \ selected \ from \ eq. \ 15.11$

Pole spanning coefficient $\alpha_{i}$ depends on the tooth saturation factor $1+K_{st}$

Now we can consider $1+K_{st}=1.4$, with {alpha_i=0.729} $K_{f}=1.085$

$$Pole flux= \phi =\alpha_{i}*\tau*L*B_{g}$$

In [36]:
B_g=0.72 
alpha_i=0.729 # Pole spanning coefficient, Fig. 14.13
flux_pole=alpha_i*tau*L*B_g #flux per pole from eq. 15.10
print flux_pole

0.0671240328061


In [37]:
K_f=1.085
W_1=ke*Vph/(4*K_f*K_w1*freq*flux_pole) # Number of turns per phase from eq. 15.12 
print W_1

70.300660275


####The number of conductor paths in parallel is defined as:
$$n_{s}= \frac{a_{1}*W_{1}}{p1*q}$$
####where;
$a_{1}=1(in \ this \ case)$

$W_{1}=70 \ (number \ of \ turns \ per \ phase)$

$p1=2 \ (number \ of \ pole \ pairs)$

$q=4$



In [38]:
a1=1
ns=a1*W_1/(pp*q) # Number of conductors per slot
print ns

8.78758253438


In [39]:
ns=8 #ns should be an even number because there are two distinct coils per slot in a double layer winding
W_1=pp*q*ns #recalculate the actual number of turns per phase
print W_1

64


In [40]:
B_g=B_g*70.30/64 #recalculate the actual air-gap flux density
print B_g

0.790875


In [41]:
I_rated=Pn/(3*Vln*pf*eff) #rated current from eq. 15.16
print I_rated
J_cos=6 #current density value selected from eq. 15.17
A_co=I_rated/(J_cos*a1) #magnetic wire cross section in mm^2 (eq. 15.18)
print A_co
d_co=sqrt(4*A_co/pi)
print d_co

66.5801125204
11.0966854201
3.75882144991


Since $d_{co}$ value is bigger than 1.3 mm, choose $a_{p}=2$

In [42]:
ap=2 #number of conductors in parallel
d_co=sqrt(4*A_co/pi*ap) 
print d_co

5.315776273


In order to meet the requirements we selected **AWG3** wire which has diameter of $5.827 \ mm$ 

- ##Stator Slot Sizing

In [43]:
K_fill=0.44 #fill factor for round-type wire
A_useful=pi*d_co**2*ap*ns/(4*K_fill) 
print A_useful


807.031666914


We choose **rectangular shape** for the stator slots among the recommended geometries.

Also we assume that all flux passes through the stator teeth. Therefore:

$$B_{g}* \tau_{s}*L = B_{ts}*b_{ts}*L*K_{Fe} $$

In [44]:
Kfe=0.96 #taking lamination insulation thickness into account 
B_ts=1.55 #tooth flux density determined from 1.5-1.65T range
b_ts=B_g*tau_s*L/(B_ts*L*Kfe) #stator tooth width 
print b_ts
b_os=2.2e-3 #shown detailed in fig. 15.5
h_os=1e-3 #shown detailed in fig. 15.5
h_w=1.5e-3 #shown detailed in fig. 15.5
b_s1=pi*(Dis+2*h_os+2*h_w)/Ns-b_ts #lower slot width in (m) 
print b_s1

0.0133865498774
0.0121269629484


From the selected geometry,useful area $A_{su}$ can be defined as follows:
$$A_{su}=h_{s}\frac{(b_{s1}+b_{s2})}{2}$$

In [54]:
b_s2=sqrt(4*A_useful*1e-6*tan(pi/Ns)+b_s1**2) #upper slot width in (m)
print b_s2
hs=2*A_useful*1e-6/(b_s1+b_s2) #Slot useful height in (m)
print hs
H_ts=1760 #magnetic field intensity for previosly selected B_ts=1.55 T (adopted from the table 15.4)
F_mts=H_ts*(hs+h_os+h_w) #stator tooth mmf 
print F_mts
F_mg=1.2*g*B_g/(4*pi*1e-7) #air-gap mmf 
print F_mg

0.0189379468411
0.0519577666495
95.8456693031
557.05397408


In [55]:
K_st=1.4-1.0 
F_mtr=K_st*F_mg-F_mts #rotor tooth mmf 
print F_mtr

126.975920329


Our calculated $B_{g}$ value is proper to satisfy the condition:

$$F_{mtr} \approx F_{mts}$$

**Note:** If  $F_{mtr}<<F_{mts}$ (or negative) it would mean that for given $1+K_{st}$  , a smaller value of flux density $B_{g}$ is required. 


In [56]:
h_cs=(Dout-(Dis+2*(h_os+h_w+hs)))/2 #stator back iron height 
print h_cs
B_cs=flux_pole/(2*L*h_cs) #back core flux density 
print B_cs

0.0634702570233
1.249697101


Our calculated **Back-core flux density**$\approx$ 1.25 T------>**this can be considered as reasonable value** 

- ##Rotor Construction